In [ ]:
################################UWCNN - https://github.com/saeed-anwar/UWCNN/tree/master/TestCode

In [1]:
##########utils.py

"""
Scipy version > 0.18 is needed, due to 'mode' option from scipy.misc.imread function
"""

import os
import glob
import h5py
import random
import matplotlib.pyplot as plt

from PIL import Image  # for loading images as YCbCr format
import scipy.misc
import scipy.ndimage
import numpy as np
import imageio

import tensorflow as tf

# FLAGS = tf.app.flags.FLAGS

def transform(images):
  return np.array(images)/127.5 - 1.
def inverse_transform(images):
  return (images+1.)/2
def prepare_data(sess, dataset):
  """
  Args:
    dataset: choose train dataset or test dataset
    
    For train dataset, output data would be ['.../t1.bmp', '.../t2.bmp', ..., '.../t99.bmp']
  
  """

  filenames = os.listdir(dataset)
  data_dir = os.path.join(os.getcwd(), dataset)
  data = glob.glob(os.path.join(data_dir, "*.bmp"))
  data = data + glob.glob(os.path.join(data_dir, "*.jpg"))
  return data

def imread(path, is_grayscale=False):
  """
  Read image using its path.
  Default value is gray-scale, and image is read by YCbCr format as the paper said.
  """
  if is_grayscale:
    return scipy.misc.imread(path, flatten=True).astype(np.float32)
  else:
    return scipy.misc.imread(path).astype(np.float32)

    
def imsave(image, path):

  imsaved = (inverse_transform(image)).astype(np.float)
  #return scipy.misc.imsave(path, imsaved)
  return imageio.imwrite(path, imsaved)

def get_image(image_path,is_grayscale=False):
  image = imread(image_path, is_grayscale)

  return transform(image)

def get_lable(image_path,is_grayscale=False):
  image = imread(image_path, is_grayscale)
  return image/255.
def imsave_lable(image, path):
  #return scipy.misc.imsave(path, image*255)
  return imageio.imwrite(path, image * 255)

In [2]:
## ops.py

import math
import numpy as np 
import tensorflow as tf


# from tensorflow.python.framework import ops

# from utils import *


try:
    image_summary = tf.image_summary
    scalar_summary = tf.scalar_summary
    histogram_summary = tf.histogram_summary
    merge_summary = tf.merge_summary
    SummaryWriter = tf.train.SummaryWriter
except:
    image_summary = tf.compat.v1.summary.image
    scalar_summary = tf.compat.v1.summary.scalar
    histogram_summary = tf.compat.v1.summary.histogram
    merge_summary = tf.compat.v1.summary.merge
    SummaryWriter = tf.compat.v1.summary.FileWriter

if "concat_v2" in dir(tf):
  def concat(tensors, axis, *args, **kwargs):
    return tf.concat_v2(tensors, axis, *args, **kwargs)
else:
  def concat(tensors, axis, *args, **kwargs):
    return tf.concat(tensors, axis, *args, **kwargs)

class batch_norm(object):
  def __init__(self, epsilon=1e-5, momentum = 0.9, name="batch_norm"):

    with tf.compat.v1.variable_scope(name):
      self.epsilon  = epsilon
      self.momentum = momentum
      self.name = name

  def __call__(self, x, train=True):
    return tf.compat.v1.contrib.layers.batch_norm(x,
                      decay=self.momentum, 
                      updates_collections=None,
                      epsilon=self.epsilon,
                      scale=True,
                      is_training=train,
                      scope=self.name)

def conv_cond_concat(x, y):
  """Concatenate conditioning vector on feature map axis."""
  x_shapes = x.get_shape()
  y_shapes = y.get_shape()
  return concat([
    x, y*tf.ones([x_shapes[0], x_shapes[1], x_shapes[2], y_shapes[3]])], 3)

def conv2d(input_, output_dim, 
       k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, padding='SAME',
       name="conv2d"):
  with tf.compat.v1.variable_scope(name):

    w = tf.compat.v1.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
              initializer=tf.compat.v1.truncated_normal_initializer(stddev=stddev))
    conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding=padding)

    biases = tf.compat.v1.get_variable('biases', [output_dim], initializer=tf.compat.v1.constant_initializer(0.0))
    conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

    return conv
def conv2d_xavier(input_, output_dim, 
       k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, padding='SAME',
       name="conv2d"):
  with tf.compat.v1.variable_scope(name):

    w = tf.compat.v1.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
              initializer=tf.compat.v1.contrib.layers.xavier_initializer_conv2d())
    conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding=padding)

    biases = tf.get_variable('biases', [output_dim], initializer=tf.compat.v1.constant_initializer(0.0))
    conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

    return conv

def deconv2d(input_, output_shape,
       k_h=3, k_w=3, d_h=1, d_w=1, stddev=0.02,
       name="deconv2d", with_w=False):
  with tf.compat.v1.variable_scope(name):
    # filter : [height, width, output_channels, in_channels]
    w = tf.compat.v1.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
              initializer=tf.compat.v1.random_normal_initializer(stddev=stddev))
    
    try:
      deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                strides=[1, d_h, d_w, 1])

    # Support for verisons of TensorFlow before 0.7.0
    except AttributeError:
      deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape,
                strides=[1, d_h, d_w, 1])

    biases = tf.compat.v1.get_variable('biases', [output_shape[-1]], initializer=tf.compat.v1.constant_initializer(0.0))
    deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

    if with_w:
      return deconv, w, biases
    else:
      return deconv
 #leaky ReLU 
def lrelu(x, leak=0.2, name="lrelu"):
  return tf.maximum(x, leak*x)

def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
  shape = input_.get_shape().as_list()

  with tf.compat.v1.variable_scope(scope or "Linear"):
    matrix = tf.compat.v1.get_variable("Matrix", [shape[1], output_size], tf.float32,
                 tf.compat.v1.random_normal_initializer(stddev=stddev))
    bias = tf.compat.v1.get_variable("bias", [output_size],
      initializer=tf.compat.v1.constant_initializer(bias_start))
    if with_w:
      return tf.matmul(input_, matrix) + bias, matrix, bias
    else:
      return tf.matmul(input_, matrix) + bias
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def conv(inputs, kernel_size, output_num, stride_size=1, init_bias=0.0, conv_padding='SAME', stddev=0.01,
         activation_func=tf.nn.relu):
    input_size = inputs.get_shape().as_list()[-1]
    conv_weights = tf.Variable(
        tf.random_normal([kernel_size, kernel_size, input_size, output_num], dtype=tf.float32, stddev=stddev),
        name='weights')
    conv_biases = tf.Variable(tf.constant(init_bias, shape=[output_num], dtype=tf.float32), 'biases')
    conv_layer = tf.nn.conv2d(inputs, conv_weights, [1, stride_size, stride_size, 1], padding=conv_padding)
    conv_layer = tf.nn.bias_add(conv_layer, conv_biases)
    if activation_func:
        conv_layer = activation_func(conv_layer)
    return conv_layer

def fc(inputs, output_size, init_bias=0.0, activation_func=tf.nn.relu, stddev=0.01):
    input_shape = inputs.get_shape().as_list()
    if len(input_shape) == 4:
        fc_weights = tf.Variable(
            tf.random_normal([input_shape[1] * input_shape[2] * input_shape[3], output_size], dtype=tf.float32,
                             stddev=stddev),
            name='weights')
        inputs = tf.reshape(inputs, [-1, fc_weights.get_shape().as_list()[0]])
    else:
        fc_weights = tf.Variable(tf.random_normal([input_shape[-1], output_size], dtype=tf.float32, stddev=stddev),
                                 name='weights')

    fc_biases = tf.Variable(tf.constant(init_bias, shape=[output_size], dtype=tf.float32), name='biases')
    fc_layer = tf.matmul(inputs, fc_weights)
    fc_layer = tf.nn.bias_add(fc_layer, fc_biases)
    return fc_layer


def lrn(inputs, depth_radius=2, alpha=0.0001, beta=0.75, bias=1.0):
    return tf.nn.local_response_normalization(inputs, depth_radius=depth_radius, alpha=alpha, beta=beta, bias=bias)

In [3]:
# from utils import ( 
# imsave,
# prepare_data
#)

import time
import os
import matplotlib.pyplot as plt
import re
import numpy as np
import tensorflow as tf
import scipy.io as scio
import cv2 as cv
# from ops import *

class T_CNN(object):

  def __init__(self, 
               sess, 
               image_height=230,
               image_width=310,
               label_height=230, 
               label_width=310,
               batch_size=1,
               c_dim=3, 
               checkpoint_dir=None, 
               sample_dir=None,
               test_image_name = None,
               id = None
               ):

    self.sess = sess
    self.is_grayscale = (c_dim == 1)
    self.image_height = image_height
    self.image_width = image_width
    self.label_height = label_height
    self.label_width = label_width
    self.batch_size = batch_size
    self.dropout_keep_prob=0.5
    self.test_image_name = test_image_name
    self.id = id
    self.c_dim = c_dim
    self.df_dim = 64
    self.checkpoint_dir = checkpoint_dir
    self.sample_dir = sample_dir

    self.build_model()

  def build_model(self):
    self.images = tf.compat.v1.placeholder(tf.float32, [self.batch_size, self.image_height, self.image_width, self.c_dim], name='images')
    self.pred_h = self.model()


    self.saver = tf.compat.v1.train.Saver()
    
  def train(self, config):


    # Stochastic gradient descent with the standard backpropagation,var_list=self.model_c_vars
    # image_test =  get_image(self.test_image_name,is_grayscale=False)
    image_testm = cv.imread(self.test_image_name, cv.IMREAD_COLOR)
    image_test = cv.cvtColor(image_testm, cv.COLOR_BGR2RGB) / 255
    shape = image_test.shape
    expand_test = image_test[np.newaxis,:,:,:]
    expand_zero = np.zeros([self.batch_size-1,shape[0],shape[1],shape[2]])
    batch_test_image = np.append(expand_test,expand_zero,axis = 0)

    tf.compat.v1.global_variables_initializer().run()
    
    
    counter = 0
    start_time = time.time()

    if self.load(self.checkpoint_dir):
      print(" [*] Load SUCCESS")
    else:
      print(" [!] Load failed...")
    result_h = self.sess.run(self.pred_h, feed_dict={self.images: batch_test_image})

    _,h ,w , c = result_h.shape
    for id in range(0,1):

        result_h0 = result_h[id,:,:,:].reshape(h , w , 3)
        #image_path0 = os.path.join(os.getcwd(), config.sample_dir)
        image_path0 = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/sample'
        final = (result_h0+1.)/2 
        output_name = os.path.basename(self.test_image_name)
        print(image_path0, output_name)
        #image_path = os.path.join(image_path0, "%2dtest_dehaze.bmp"%(self.id))
        image_path = os.path.join(image_path0, output_name)
        imsave_lable(final, image_path)


  def model(self):

    with tf.compat.v1.variable_scope("model_h") as scope:
        if self.id > 0: 
          scope.reuse_variables()
        image_conv1 = tf.nn.relu(conv2d(self.images, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze1"))
        image_conv2 = tf.nn.relu(conv2d(image_conv1, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze2"))
        image_conv3 = tf.nn.relu(conv2d(image_conv2, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze3"))
        dehaze_concat1 = tf.concat(axis = 3, values = [image_conv1,image_conv2,image_conv3,self.images])
        image_conv4 = tf.nn.relu(conv2d(dehaze_concat1, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze4"))
        image_conv5 = tf.nn.relu(conv2d(image_conv4, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze5"))
        image_conv6 = tf.nn.relu(conv2d(image_conv5, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze6"))
        dehaze_concat2 = tf.concat(axis = 3, values = [dehaze_concat1,image_conv4,image_conv5,image_conv6])
        image_conv7 = tf.nn.relu(conv2d(dehaze_concat2, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze7"))
        image_conv8 = tf.nn.relu(conv2d(image_conv7, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze8"))
        image_conv9 = tf.nn.relu(conv2d(image_conv8, 16, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze9"))
        dehaze_concat3 = tf.concat(axis = 3, values = [dehaze_concat2,image_conv7,image_conv8,image_conv9])
        image_conv10 = conv2d(dehaze_concat3, 3, k_h=3, k_w=3, d_h=1, d_w=1,name="conv2d_dehaze10")
        out = tf.add(self.images , image_conv10)
    return out

  def save(self, checkpoint_dir, step):
    model_name = "coarse.model"
    model_dir = "%s_%s" % ("coarse", self.label_height)
    checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    self.saver.save(self.sess,
                    os.path.join(checkpoint_dir, model_name),
                    global_step=step)

  def load(self, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    model_dir = "%s_%s" % ("coarse", self.label_height)
    checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
        return True
    else:
        return False

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# from model import T_CNN
# from utils import *
import numpy as np
import tensorflow as tf
import cv2 as cv

import pprint
import os
from absl import flags

# flags = tf.app.flags
flags.DEFINE_integer("epoch", 120, "Number of epoch [120]")
flags.DEFINE_integer("batch_size", 1, "The size of batch images [128]")
flags.DEFINE_integer("image_height", 230, "The size of image to use [230]")
flags.DEFINE_integer("image_width", 310, "The size of image to use [310]")
flags.DEFINE_integer("label_height", 230, "The size of label to produce [230]")
flags.DEFINE_integer("label_width", 310, "The size of label to produce [310]")
flags.DEFINE_float("learning_rate", 0.0001, "The learning rate of gradient descent algorithm [1e-4]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("c_dim", 3, "Dimension of image color. [3]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Name of checkpoint directory [checkpoint]")
flags.DEFINE_string("sample_dir", "sample", "Name of sample directory [sample]")
flags.DEFINE_string("test_data_dir", "test", "Name of sample directory [test]")
flags.DEFINE_boolean("is_train", False, "True for training, False for testing [True]")
FLAGS = flags.FLAGS

pp = pprint.PrettyPrinter()

def main(_):
  pp.pprint(flags.FLAGS.__flags)

  FLAGS.checkpoint_dir = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/checkpoint'
  if not os.path.exists(FLAGS.checkpoint_dir):
    os.makedirs(FLAGS.checkpoint_dir)
  
  FLAGS.sample_dir = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/sample'
  if not os.path.exists(FLAGS.sample_dir):
    os.makedirs(FLAGS.sample_dir)

  
  #filenames = os.listdir('test_real')
  #data_dir = os.path.join(os.getcwd(), 'test_real')
  filenames = os.listdir('/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/raw/')
  data_dir = os.path.join(os.getcwd(),'/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/raw/')
  data = glob.glob(os.path.join(data_dir, "*.bmp"))
  test_data_list = data + glob.glob(os.path.join(data_dir, "*.png"))

  for ide in range(0,len(test_data_list)):
    #image_test =  get_image(test_data_list[ide],is_grayscale=False)
    image_test = cv.imread(test_data_list[ide], 1)
    shape = image_test.shape
    with tf.compat.v1.Session() as sess:
      # with tf.device('/cpu:0'):
        srcnn = T_CNN(sess, 
                  image_height=shape[0],
                  image_width=shape[1],  
                  label_height=FLAGS.label_height, 
                  label_width=FLAGS.label_width, 
                  batch_size=FLAGS.batch_size,
                  c_dim=FLAGS.c_dim, 
                  checkpoint_dir=FLAGS.checkpoint_dir,
                  sample_dir=FLAGS.sample_dir,
                  test_image_name = test_data_list[ide],
                  id = ide
                  )

        srcnn.train(FLAGS)
        sess.close()
if __name__ == '__main__':
  # tf.app.run()
  tf.compat.v1.app.run()

{'?': <absl.app.HelpFlag object at 0x7f5597d96090>,
 'alsologtostderr': <absl.flags._flag.BooleanFlag object at 0x7f55e0e4e550>,
 'batch_size': <absl.flags._flag.Flag object at 0x7f5597d962d0>,
 'beta1': <absl.flags._flag.Flag object at 0x7f5597d96b10>,
 'c_dim': <absl.flags._flag.Flag object at 0x7f5597d96a50>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x7f5597d96c50>,
 'epoch': <absl.flags._flag.Flag object at 0x7f5597d960d0>,
 'hbm_oom_exit': <absl.flags._flag.BooleanFlag object at 0x7f55a26307d0>,
 'help': <absl.app.HelpFlag object at 0x7f5597d96090>,
 'helpfull': <absl.app.HelpfullFlag object at 0x7f55a81c4b90>,
 'helpshort': <absl.app.HelpshortFlag object at 0x7f5597d96390>,
 'helpxml': <absl.app.HelpXMLFlag object at 0x7f55a81c4b50>,
 'image_height': <absl.flags._flag.Flag object at 0x7f5597d967d0>,
 'image_width': <absl.flags._flag.Flag object at 0x7f5597d96ad0>,
 'is_train': <absl.flags._flag.BooleanFlag object at 0x7f5597d96fd0>,
 'label_height': <absl.flags._flag.F

I0921 14:41:54.687045 140007564044160 saver.py:1298] Restoring parameters from /content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/checkpoint/coarse_230/coarse.model-typeII


 [*] Load SUCCESS


W0921 14:42:26.460001 140007564044160 util.py:61] Lossy conversion from float32 to uint8. Range [37.78368377685547, 362.71600341796875]. Convert image to uint8 prior to saving to suppress this warning.


/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/sample 125_img_.png
 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/checkpoint/coarse_230/coarse.model-typeII


...

 [*] Load SUCCESS


W0921 14:51:47.129258 140007564044160 util.py:61] Lossy conversion from float32 to uint8. Range [90.8766098022461, 278.8345031738281]. Convert image to uint8 prior to saving to suppress this warning.


/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/sample 18_img_.png


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
###### FULL-REFERENCE IMAGE QUALITY ASSESSMENT IN TERMS OF MSE, PSNR, AND SSIM #######

import numpy as np
import cv2
import os

from skimage.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity 


target_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/reference'
predicted_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/OtherStateOfArtResults/UWCNN/sample'


im_h = 224
im_w = 224

sumMSE = 0;
sumPSNR = 0;
sumSSIM = 0;
count = 0;


for image_name in os.listdir(target_path):

    #read the raw images
    target_img = cv2.imread(os.path.join(target_path, image_name))
    predicted_img = cv2.imread(os.path.join(predicted_path, image_name))
    

    #resize the images into 224 x 224
    target_img = cv2.resize(target_img, (im_h, im_w))
    predicted_img = cv2.resize(predicted_img, (im_h, im_w))


    #calculate the MSE, PSNR, and SSIM 
    MSE = mean_squared_error(target_img, predicted_img)
    PSNR = peak_signal_noise_ratio(target_img, predicted_img)
    SSIM = structural_similarity(target_img, predicted_img, multichannel=True)

    sumMSE = sumMSE + MSE
    sumPSNR = sumPSNR + PSNR
    sumSSIM = sumSSIM + SSIM
    count = count + 1
    
    # print('SSIM: ', SSIM)
    # print('MSE: ', MSE)
    # print('PSNR: ', PSNR)


# print('sumMSE: ',sumMSE)
# print('sumPSNR: ', sumPSNR)
# print('sumSSIM: ', sumSSIM)
# print('count: ', count)


# Get the Average value 
mse_value = sumMSE/count
psnr_value = sumPSNR/count
ssim_value = sumSSIM/count


print("MSE for testing data :", format(mse_value, ".4f"))
print("PSNR for testing data :", format(psnr_value, ".4f"), "dB")
print("SSIM for testing data :", format(ssim_value, ".4f"))

MSE for testing data : 4288.3160
PSNR for testing data : 12.3526 dB
SSIM for testing data : 0.5351
